# In this notebook we will use ChatGPT to execute Python code (mostly Pandas) to answer questions on the titanic dataset. We will do this without the use of any agents (e.g. langchain, transformer agents)

## Let's start with loading the titanic datasets:

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
test = pd.read_csv('test.csv')
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


# Ready to prepare ChatGPT:

In [4]:
from dotenv import load_dotenv
dotenv_path = "/Users/aviad.atlas/PycharmProjects/aviad_research/langchain/.env"
load_dotenv(dotenv_path)

import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
intro = "I have the following tables. Each table I will present to you includes the table header and a sample of up to 5 rows of the table. Before the sample I will specify the name of the table. Here are the tables: "
tables = {'train': train.head().to_string(), 'test': test.head().to_string()}

tables_info = "\n\n"
for table_name, table in tables.items():
    tables_info += table_name + table + "\n\n"

In [6]:
def run_via_chatgpt(user_instruction, history):
    print('The user instruction:\n', user_instruction)
    full_instruction = f"""Now here's what I want you to do. Write me pandas code that does the following "{user_instruction}". Assume that each table is stored in a pandas dataframe, the dataframe name is the same as the table name provided. 
    If you use a package other than pandas (for example numpy), add the import code to the response. In your response only output the desired pandas code, no explanation or example.
    Add a ; between the lines of code, so all pieces of code can be run together in one execution. Do not add an enter character, \n, for a piece of code that should run solely, just have it in the same line.
    Call the result `result`. If more than one result is actually asked for, store the first result in `result`, the second in `result2`, the third in `result3`, and so on. """
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": history + full_instruction,
        }
    ],
    )
    res = completion.choices[0].message.content.lstrip()
    print('\n')
    print('The code to run:\n', res)
    new_history = history + full_instruction + "\n" + res
    code_chunks_to_run = [code for code in res.split('\n') if '```' not in code]
    return code_chunks_to_run, new_history

## Ready to query:

In [7]:
history = intro + tables_info

user_instruction = """In the train set, how many rows are missing the Cabin value?"""

In [8]:
code_chunks_to_run, new_history = run_via_chatgpt(user_instruction, history)

The user instruction:
 In the train set, how many rows are missing the Cabin value?


The code to run:
 result = train['Cabin'].isna().sum();


### Now we got our Python code to execute. We can now answer our question:

In [9]:
for code in code_chunks_to_run:
    exec(code)

In [10]:
print(user_instruction, "\nChatGPT's answer is: ", result)

In the train set, how many rows are missing the Cabin value? 
ChatGPT's answer is:  687


## Let's try a few more queries:

In [11]:
user_instruction = """In the test set, what is the average fare price?"""

In [12]:
code_chunks_to_run, new_history = run_via_chatgpt(user_instruction, new_history)

The user instruction:
 In the test set, what is the average fare price?


The code to run:
 result = test['Fare'].mean();


In [13]:
for code in code_chunks_to_run:
    exec(code)

In [14]:
print(user_instruction, "\nChatGPT's answer is: ", result)

In the test set, what is the average fare price? 
ChatGPT's answer is:  35.627188489208635


In [15]:
user_instruction = """For the test set, what is the distribution of the Embarked value?"""

In [16]:
code_chunks_to_run, new_history = run_via_chatgpt(user_instruction, new_history)

The user instruction:
 For the test set, what is the distribution of the Embarked value?


The code to run:
 result = test['Embarked'].value_counts();


In [17]:
for code in code_chunks_to_run:
    exec(code)

In [18]:
print(user_instruction, "\nChatGPT's answer is: \n", result)

For the test set, what is the distribution of the Embarked value? 
ChatGPT's answer is: 
 Embarked
S    270
C    102
Q     46
Name: count, dtype: int64


## This is pretty cool
## Now let's try to perfrom some actions involving both datasets:

In [19]:
user_instruction = """Give me for each set only the rows that the class is 3"""

In [20]:
code_chunks_to_run, new_history = run_via_chatgpt(user_instruction, new_history)

The user instruction:
 Give me for each set only the rows that the class is 3


The code to run:
 result = train[train['Pclass']==3];
result2 = test[test['Pclass']==3]


In [21]:
for code in code_chunks_to_run:
    exec(code)

In [22]:
result

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
882,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [23]:
result2

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
412,1304,3,"Henriksson, Miss. Jenny Lovisa",female,28.0,0,0,347086,7.7750,NaN,S
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [24]:
user_instruction = """Create a single dataset that has the train data and then the test data"""

In [25]:
code_chunks_to_run, new_history = run_via_chatgpt(user_instruction, new_history)

The user instruction:
 Create a single dataset that has the train data and then the test data


The code to run:
 result = pd.concat([train, test], axis=0, sort=False);


In [29]:
for code in code_chunks_to_run:
    exec(code)

In [30]:
result

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [43]:
user_instruction = """I want to check if the passenger id is unique. List ids that appear dupliacte times, whether in the train or test"""

In [44]:
code_chunks_to_run, new_history = run_via_chatgpt(user_instruction, new_history)

The user instruction:
 I want to check if the passenger id is unique. List ids that appear dupliacte times, whether in the train or test


The code to run:
 result = pd.concat([train['PassengerId'], test['PassengerId']])[pd.concat([train['PassengerId'], test['PassengerId']]).duplicated()].tolist()


In [41]:
for code in code_chunks_to_run:
    exec(code)

In [42]:
result

array([], dtype=int64)